<a href="https://colab.research.google.com/github/jaketerrito/speedchallenge/blob/autoencoder/Feedforward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/jaketerrito/speedchallenge/raw/master/data/train.mp4
!wget https://github.com/jaketerrito/speedchallenge/raw/master/data/train.txt

In [2]:
#!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2 as cv

In [3]:
import numpy as np
import time
from os.path import splitext

def denseflow(fname, resize_factor):
    cap = cv.VideoCapture(fname)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
        exit()

    # Get the video dimensions, 3 is the ordinal value of CV_CAP_PROP_FRAME_WIDTH, 4 is CV_CAP_PROP_FRAME_HEIGHT
    # Also resize them because these images are too big
    width = int(cap.get(3) / resize_factor)
    height = int(cap.get(4) / resize_factor)
    shape = (width,height)
    size = int(cap.get(7))
    ret, frame1 = cap.read()
    frame1 = cv.resize(frame1,shape)
    prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255

    frames = np.zeros((size,hsv.shape[0],hsv.shape[1],3))
    frames[0] = hsv

    fcount = 1
    t = time.time()

    while(1):
        ret, frame2 = cap.read()
        if not ret:
            break
        frame2 = cv.resize(frame2,shape)
        next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
        flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
        frames[fcount] = hsv
        prvs = next
        fcount += 1
    cap.release()

    f, ext = splitext(fname)
    #np.savez(f+'_op', frames)
    return frames

def plainVideo(fname, resize_factor):
    cap = cv.VideoCapture(fname)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
        exit()

    width = int(cap.get(3) / resize_factor)
    height = int(cap.get(4) / resize_factor)
    shape = (width,height)
    size = int(cap.get(7))
    ret, frame1 = cap.read()
    frame1 = cv.resize(frame1,shape)
    
    frames = np.zeros((size, frame1.shape[0], frame1.shape[1],3))
    frames[0] = frame1
    fcount = 1
    while(1):
        ret, frame2 = cap.read()
        if not ret:
            break
        frame2 = cv.resize(frame2,shape)
        frames[fcount] = frame2
        fcount += 1
    cap.release()
    return frames

In [4]:
fname = "data/train.mp4"

try:
    f, ext = splitext(fname)
    with np.load(f + '_op.npz') as data:
        video = data['arr_0']
except:
    print("Could not find preprocessed video, creating it now")
    video = denseflow(fname, 4)

#video = plainVideo(fname, 4)
width = video.shape[2]
height = video.shape[1]
video_size = len(video)

video = video/255

In [5]:
print("Processing speeds.")
with open('data/train.txt') as f:
	speeds = f.readlines()
	speeds = np.array([float(x.strip()) for x in speeds])
	speeds = speeds

Processing speeds.


In [50]:
import matplotlib.pyplot as plt
import random
from tensorflow.keras.layers import Input, Dense, Flatten, Conv3D, MaxPooling3D, MaxPooling2D, BatchNormalization, Dropout, LeakyReLU, PReLU, Conv2DTranspose, Reshape, ZeroPadding2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
import tensorflow as tf

In [7]:
'''
A Generator that produces sets of training features and labels
* video:  A list of video frames
* speeds: A list of corresponding speeds
* batch_size: number of samples to produce for each batch
* sequence_length: number of images that model consider for each prediction
'''
def image_generator(video, batch_size, sequence_length):
    while True:
        sequences = []
        #speed_vals = []
        next_frames = []
        while len(sequences) < batch_size:
            frame_num = random.randrange(sequence_length,len(video)-1)
            sequence = video[frame_num-sequence_length:frame_num]
            next_frame = video[frame_num-1]
            '''
            flip = random.choice([True,False])
            angle = random.uniform(-20,20)
            scale = random.uniform(.8,1.2)
            for i, image in enumerate(sequence):
                # Augmentation
                image = skimage.transform.rescale(image, scale=scale)
                image = skimage.transform.resize(image, output_shape=sequence[i].shape, mode='constant')
                image = skimage.transform.rotate(image, angle=angle)
                # Really need to see the types of values before we add this noise
                image = image + np.random.normal(scale=.5,size=image.shape)
                sequence[i] = image
            '''
            sequences.append(sequence)
            next_frames.append(next_frame)
        yield np.array(sequences), np.array(next_frames)

def speed_generator(video, speeds, batch_size, sequence_length):
    while True:
        sequences = []
        speed_vals = []
        while len(sequences) < batch_size:
            frame_num = random.randrange(sequence_length,len(video))
            sequence = video[frame_num-sequence_length:frame_num]
            '''
            flip = random.choice([True,False])
            angle = random.uniform(-20,20)
            scale = random.uniform(.8,1.2)
            for i, image in enumerate(sequence):
                # Augmentation
                image = skimage.transform.rescale(image, scale=scale)
                image = skimage.transform.resize(image, output_shape=sequence[i].shape, mode='constant')
                image = skimage.transform.rotate(image, angle=angle)
                # Really need to see the types of values before we add this noise
                image = image + np.random.normal(scale=.5,size=image.shape)
                sequence[i] = image
            '''
            sequences.append(sequence)
            speed_vals.append(speeds[frame_num-1])
        yield np.array(sequences), np.array(speed_vals)

In [94]:
batch_size = 16
sequence_length = 8
epochs = 10
split = .8
latent_size = 16

train_ae_gen = image_generator(video[:int(video_size*split)], batch_size, sequence_length)
val_ae_gen = image_generator(video[int(video_size*split):], batch_size, sequence_length)

train_re_gen = speed_generator(video[:int(video_size*split)], speeds[:int(video_size*split)], batch_size, sequence_length)
val_re_gen = speed_generator(video[int(video_size*split):], speeds[int(video_size*split):], batch_size, sequence_length)

# Will return a feature and label set.	
# Features are a list of image sequences in the form: (sequence_length, img_height, img_width, dimensions)
def get_encoder():
    inputs = Input((sequence_length,height,width,3))

    x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(inputs)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv3D(32,(3,3,3),strides=(2,2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv3D(32,(3,3,3),strides=(2,2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dropout(.5)(x)
    
    x = Dense(32,activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    outputs = Dense(latent_size,activation='tanh')(x)
    return Model(inputs=inputs,outputs=outputs)
  
def get_decoder():
    inputs = Input((latent_size,))
    x = Dense(32,activation=None)(inputs)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dense(14*19*32,activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Reshape((14,19,32))(x)
    x = Conv2DTranspose(32,(3,3),strides=(2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x) 
    x = Conv2DTranspose(32,(3,3),strides=(2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(3,(3,3),strides=(2,2),activation='sigmoid')(x)
    #x = LeakyReLU(alpha=0.1)(x)
    x = ZeroPadding2D(((0,1),(0,1)))(x)
    outputs = BatchNormalization()(x)
    return Model(inputs=inputs, outputs=outputs,name='decoder')
# regularizers instead of batch norm?
# try increasing dropout?
# lowering image resolution?
# Try predicting the latent vector of the sequence+nextframe
def get_regression():
    inputs = Input((latent_size,))
    x = Dropout(.3)(inputs)
    x = Dense(16,activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dropout(.3)(x)
    outputs = Dense(1,activation='linear')(x)
    return Model(inputs=inputs,outputs=outputs,name='regression')

get_encoder().summary()
get_decoder().summary()
#get_regression().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_119 (InputLayer)       (None, 8, 120, 160, 3)    0         
_________________________________________________________________
conv3d_103 (Conv3D)          (None, 8, 59, 79, 32)     896       
_________________________________________________________________
leaky_re_lu_287 (LeakyReLU)  (None, 8, 59, 79, 32)     0         
_________________________________________________________________
batch_normalization_189 (Bat (None, 8, 59, 79, 32)     128       
_________________________________________________________________
conv3d_104 (Conv3D)          (None, 3, 29, 39, 32)     27680     
_________________________________________________________________
leaky_re_lu_288 (LeakyReLU)  (None, 3, 29, 39, 32)     0         
_________________________________________________________________
batch_normalization_190 (Bat (None, 3, 29, 39, 32)     128       
__________

In [ ]:
# build the autoencoder
inputs = Input((sequence_length,height,width,3))
latent = get_encoder()(inputs)
regression_output = get_regression()(latent)
dec_output = get_decoder()(latent)
autoencoder = Model(inputs=inputs,outputs=dec_output)
encoder = Model(inputs=inputs, outputs=latent)
regression = Model(inputs=inputs,outputs=regression_output)

# compile with mean absolute error loss on reconstructed image and binary_cross for discriminator
autoencoder.compile(loss='mse',optimizer=Adam())
regression.compile(loss='mse',optimizer=RMSprop())

In [98]:
history = []
reg_history = []
for i in range(epochs):
    print("Real epoch: ", i+1)
    thistory = autoencoder.fit_generator(
        train_ae_gen, 
        steps_per_epoch=int(video_size*split/batch_size), 
        validation_data=val_ae_gen, 
        validation_steps=int(video_size*(1-split)/batch_size),
        epochs=1,
        verbose=True,
        callbacks=[ModelCheckpoint('./data/weights.hdf5',save_best_only=True)]
    )
    print()
    history.append(thistory)
    treg_history = regression.fit_generator(
        train_re_gen, 
        steps_per_epoch=int(video_size*split/batch_size), 
        validation_data=val_re_gen, 
        validation_steps=int(video_size*(1-split)/batch_size),
        epochs=3,
        verbose=True,
        callbacks=[ModelCheckpoint('./data/reg_weights.hdf5',save_best_only=True)]
    )
    reg_history.append(treg_history)
    print()

Real epoch:  1
Epoch 1/1
1020/1020 [==============================] - 107s 105ms/step - loss: 3.9806 - val_loss: 0.7604

Epoch 1/3
1020/1020 [==============================] - 91s 89ms/step - loss: 46.1873 - val_loss: 11.4299
Epoch 2/3
1020/1020 [==============================] - 84s 83ms/step - loss: 22.6961 - val_loss: 37.4929
Epoch 3/3
1020/1020 [==============================] - 87s 86ms/step - loss: 18.9815 - val_loss: 18.4132

Real epoch:  2
Epoch 1/1
1020/1020 [==============================] - 102s 100ms/step - loss: 0.4132 - val_loss: 0.3222

Epoch 1/3
1020/1020 [==============================] - 87s 85ms/step - loss: 17.8149 - val_loss: 10.3162
Epoch 2/3
1020/1020 [==============================] - 86s 84ms/step - loss: 16.2803 - val_loss: 14.0028
Epoch 3/3
1020/1020 [==============================] - 86s 84ms/step - loss: 14.9339 - val_loss: 10.4036

Real epoch:  3
Epoch 1/1
1020/1020 [==============================] - 98s 96ms/step - loss: 0.3323 - val_loss: 0.3175

Epoch 1

In [ ]:
history = autoencoder.fit_generator(
    train_ae_gen, 
    steps_per_epoch=int(video_size*split/batch_size), 
    validation_data=val_ae_gen, 
    validation_steps=int(video_size*(1-split)/batch_size),
    epochs=epochs,
    verbose=True,
    callbacks=[ModelCheckpoint('./data/weights.hdf5',save_best_only=True)]
)

Epoch 1/10
1020/1020 [==============================] - 105s 103ms/step - loss: 3.9809 - val_loss: 0.7546
Epoch 2/10
1020/1020 [==============================] - 94s 92ms/step - loss: 0.3980 - val_loss: 0.3126
Epoch 3/10
1020/1020 [==============================] - 94s 92ms/step - loss: 0.3129 - val_loss: 0.3038
Epoch 4/10
1020/1020 [==============================] - 93s 92ms/step - loss: 0.3060 - val_loss: 0.2961
Epoch 5/10
1020/1020 [==============================] - 94s 92ms/step - loss: 0.3000 - val_loss: 0.2951
Epoch 6/10
1020/1020 [==============================] - 93s 92ms/step - loss: 0.2942 - val_loss: 0.2866
Epoch 7/10
1020/1020 [==============================] - 51360s 50s/step - loss: 0.2897 - val_loss: 0.2890
Epoch 8/10
  40/1020 [>.............................] - ETA: 1:43 - loss: 0.2829

In [ ]:
reg_history = regression.fit_generator(
    train_re_gen, 
    steps_per_epoch=int(video_size*split/batch_size), 
    validation_data=val_re_gen, 
    validation_steps=int(video_size*(1-split)/batch_size),
    epochs=epochs,
    verbose=True,
    callbacks=[ModelCheckpoint('./data/reg_weights.hdf5',save_best_only=True)]
)

In [85]:
batch_size = 16
sequence_length = 1
epochs = 10
split = .8
latent_size = 16

train_ae_gen = image_generator(video[:int(video_size*split)], batch_size, sequence_length)
val_ae_gen = image_generator(video[int(video_size*split):], batch_size, sequence_length)

# Will return a feature and label set.	
# Features are a list of image sequences in the form: (sequence_length, img_height, img_width, dimensions)
def get_encoder():
    inputs = Input((sequence_length,height,width,3))

    # A convolution being applied to each image seperately
    x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(inputs)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(32,activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    outputs = Dense(latent_size,activation='tanh')(x)
    return Model(inputs=inputs,outputs=outputs)
  
def get_decoder():
    inputs = Input((latent_size,))
    x = Dense(32,activation=None)(inputs)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dense(6*9*32,activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Reshape((6,9,32))(x)
    x = Conv2DTranspose(32,(3,3),strides=(2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = ZeroPadding2D(((1,0),(0,0)))(x)
    x = BatchNormalization()(x) 
    x = Conv2DTranspose(32,(3,3),strides=(2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(32,(3,3),strides=(2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(3,(3,3),strides=(2,2),activation='sigmoid')(x)
    #x = LeakyReLU(alpha=0.1)(x)
    x = ZeroPadding2D(((0,1),(0,1)))(x)
    outputs = BatchNormalization()(x)
    return Model(inputs=inputs, outputs=outputs,name='decoder')
# regularizers instead of batch norm?
# try increasing dropout?
# lowering image resolution?
# Try predicting the latent vector of the sequence+nextframe
def get_regression():
    inputs = Input((latent_size,))
    x = Dropout(.3)(inputs)
    x = Dense(32,activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dropout(.3)(x)
    outputs = Dense(1,activation='linear')(x)
    return Model(inputs=inputs,outputs=outputs,name='regression')

#get_encoder().summary()
#get_decoder().summary()
#get_regression().summary()

In [88]:
# build the autoencoder
inputs = Input((sequence_length,height,width,3))
latent = get_encoder()(inputs)
regression_output = get_regression()(latent)
dec_output = get_decoder()(latent)
autoencoder = Model(inputs=inputs,outputs=dec_output)
encoder = Model(inputs=inputs, outputs=latent)
regression = Model(inputs=inputs,outputs=regression_output)

# compile with mean absolute error loss on reconstructed image and binary_cross for discriminator
autoencoder.compile(loss='mse',optimizer=Adam())
regression.compile(loss='mse',optimizer=RMSprop())

In [87]:
history = []
reg_history = []
for i in range(epochs):
    print("Real epoch: ", i+1)
    thistory = autoencoder.fit_generator(
        train_ae_gen, 
        steps_per_epoch=int(video_size*split/batch_size), 
        validation_data=val_ae_gen, 
        validation_steps=int(video_size*(1-split)/batch_size),
        epochs=1,
        verbose=True,
        callbacks=[ModelCheckpoint('./data/weights.hdf5',save_best_only=True)]
    )
    print()
    history.append(thistory)
    treg_history = regression.fit_generator(
        train_re_gen, 
        steps_per_epoch=int(video_size*split/batch_size), 
        validation_data=val_re_gen, 
        validation_steps=int(video_size*(1-split)/batch_size),
        epochs=3,
        verbose=True,
        callbacks=[ModelCheckpoint('./data/reg_weights.hdf5',save_best_only=True)]
    )
    reg_history.append(treg_history)
    print()

Real epoch:  1
Epoch 1/1
1020/1020 [==============================] - 37s 37ms/step - loss: 4.4328 - val_loss: 0.6496

Epoch 1/3
1020/1020 [==============================] - 25s 24ms/step - loss: 43.3589 - val_loss: 10.1152
Epoch 2/3
1020/1020 [==============================] - 18s 18ms/step - loss: 20.5766 - val_loss: 9.4821
Epoch 3/3
1020/1020 [==============================] - 18s 18ms/step - loss: 16.7248 - val_loss: 10.5411

Real epoch:  2
Epoch 1/1
1020/1020 [==============================] - 29s 28ms/step - loss: 0.3724 - val_loss: 0.2898

Epoch 1/3
1020/1020 [==============================] - 18s 18ms/step - loss: 16.9085 - val_loss: 7.3505
Epoch 2/3
1020/1020 [==============================] - 18s 18ms/step - loss: 12.4865 - val_loss: 14.8586
Epoch 3/3
1020/1020 [==============================] - 18s 18ms/step - loss: 11.3231 - val_loss: 10.9446

Real epoch:  3
Epoch 1/1
1020/1020 [==============================] - 29s 28ms/step - loss: 0.2729 - val_loss: 0.2688

Epoch 1/3
102

In [89]:
history = autoencoder.fit_generator(
    train_ae_gen, 
    steps_per_epoch=int(video_size*split/batch_size), 
    validation_data=val_ae_gen, 
    validation_steps=int(video_size*(1-split)/batch_size),
    epochs=epochs,
    verbose=True,
    callbacks=[ModelCheckpoint('./data/weights.hdf5',save_best_only=True)]
)

Epoch 1/10
1020/1020 [==============================] - 38s 37ms/step - loss: 4.3396 - val_loss: 0.6463
Epoch 2/10
1020/1020 [==============================] - 29s 29ms/step - loss: 0.3453 - val_loss: 0.2687
Epoch 3/10
1020/1020 [==============================] - 29s 29ms/step - loss: 0.2486 - val_loss: 0.2547
Epoch 4/10
1020/1020 [==============================] - 29s 29ms/step - loss: 0.2345 - val_loss: 0.2429
Epoch 5/10
1020/1020 [==============================] - 29s 29ms/step - loss: 0.2284 - val_loss: 0.2380
Epoch 6/10
1020/1020 [==============================] - 29s 29ms/step - loss: 0.2230 - val_loss: 0.2367
Epoch 7/10
1020/1020 [==============================] - 29s 29ms/step - loss: 0.2215 - val_loss: 0.2347
Epoch 8/10
1020/1020 [==============================] - 29s 29ms/step - loss: 0.2200 - val_loss: 0.2334
Epoch 9/10
1020/1020 [==============================] - 29s 29ms/step - loss: 0.2184 - val_loss: 0.2361
Epoch 10/10
1020/1020 [==============================] - 29s 29m

In [90]:
reg_history = regression.fit_generator(
    train_re_gen, 
    steps_per_epoch=int(video_size*split/batch_size), 
    validation_data=val_re_gen, 
    validation_steps=int(video_size*(1-split)/batch_size),
    epochs=epochs,
    verbose=True,
    callbacks=[ModelCheckpoint('./data/reg_weights.hdf5',save_best_only=True)]
)

Epoch 1/10
1020/1020 [==============================] - 25s 25ms/step - loss: 33.3662 - val_loss: 20.3352
Epoch 2/10
1020/1020 [==============================] - 18s 18ms/step - loss: 17.6701 - val_loss: 12.2127
Epoch 3/10
1020/1020 [==============================] - 18s 18ms/step - loss: 14.1765 - val_loss: 12.2445
Epoch 4/10
1020/1020 [==============================] - 18s 18ms/step - loss: 11.9220 - val_loss: 12.4837
Epoch 5/10
1020/1020 [==============================] - 18s 18ms/step - loss: 10.6350 - val_loss: 14.2083
Epoch 6/10
1020/1020 [==============================] - 18s 18ms/step - loss: 9.3798 - val_loss: 14.9232
Epoch 7/10
1020/1020 [==============================] - 18s 18ms/step - loss: 9.0205 - val_loss: 16.3534
Epoch 8/10
1020/1020 [==============================] - 18s 18ms/step - loss: 8.3712 - val_loss: 14.0255
Epoch 9/10
1020/1020 [==============================] - 18s 18ms/step - loss: 8.0452 - val_loss: 25.5787
Epoch 10/10
1020/1020 [===========================

In [70]:
batch_size = 16
sequence_length = 1
reg_seq = 3
epochs = 10
split = .8
latent_size = 16

train_ae_gen = image_generator(video[:int(video_size*split)], batch_size, sequence_length)
val_ae_gen = image_generator(video[int(video_size*split):], batch_size, sequence_length)

# Will return a feature and label set.	
# Features are a list of image sequences in the form: (sequence_length, img_height, img_width, dimensions)
def get_encoder():
    inputs = Input((sequence_length,height,width,3))

    # A convolution being applied to each image seperately
    x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(inputs)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv3D(32,(1,3,3),strides=(1,2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(32,activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    outputs = Dense(latent_size,activation='tanh')(x)
    return Model(inputs=inputs,outputs=outputs)
  
def get_decoder():
    inputs = Input((latent_size,))
    x = Dense(32,activation=None)(inputs)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dense(6*9*32,activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Reshape((6,9,32))(x)
    x = Conv2DTranspose(32,(3,3),strides=(2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = ZeroPadding2D(((1,0),(0,0)))(x)
    x = BatchNormalization()(x) 
    x = Conv2DTranspose(32,(3,3),strides=(2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(32,(3,3),strides=(2,2),activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(3,(3,3),strides=(2,2),activation='sigmoid')(x)
    #x = LeakyReLU(alpha=0.1)(x)
    x = ZeroPadding2D(((0,1),(0,1)))(x)
    outputs = BatchNormalization()(x)
    return Model(inputs=inputs, outputs=outputs,name='decoder')
# regularizers instead of batch norm?
# try increasing dropout?
# lowering image resolution?
# Try predicting the latent vector of the sequence+nextframe
def get_regression():
    inputs = Input((latent_size*reg_seq,))
    x = Dropout(.3)(inputs)
    x = Dense(32,activation=None)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dropout(.3)(x)
    outputs = Dense(1,activation='linear')(x)
    return Model(inputs=inputs,outputs=outputs,name='regression')

#get_encoder().summary()
#get_decoder().summary()
#get_regression().summary()

In [60]:
# build the autoencoder
inputs = Input((sequence_length,height,width,3))
latent = get_encoder()(inputs)
dec_output = get_decoder()(latent)
autoencoder = Model(inputs=inputs,outputs=dec_output)
encoder = Model(inputs=inputs, outputs=latent)

# compile with mean absolute error loss on reconstructed image and binary_cross for discriminator
autoencoder.compile(loss='mse',optimizer=Adam())

graph = tf.get_default_graph()

In [67]:
# build the regressor
reg_inputs = Input((reg_seq*latent_size,))
reg_outputs = get_regression()(reg_inputs)
regression = Model(inputs=reg_inputs, outputs=reg_outputs)

regression.compile(loss='mse',optimizer=RMSprop())

In [62]:
def concat_speed_generator(video, speeds, batch_size, sequence_length, ae):
    global graph
    while True:
        sequences = []
        speed_vals = []
        while len(sequences) < batch_size:
            frame_num = random.randrange(sequence_length,len(video))
            sequence = video[frame_num-sequence_length:frame_num]
            '''
            flip = random.choice([True,False])
            angle = random.uniform(-20,20)
            scale = random.uniform(.8,1.2)
            for i, image in enumerate(sequence):
                # Augmentation
                image = skimage.transform.rescale(image, scale=scale)
                image = skimage.transform.resize(image, output_shape=sequence[i].shape, mode='constant')
                image = skimage.transform.rotate(image, angle=angle)
                # Really need to see the types of values before we add this noise
                image = image + np.random.normal(scale=.5,size=image.shape)
                sequence[i] = image
            '''
            sequence = np.expand_dims(sequence, axis=1)
            with graph.as_default():
                preds = ae.predict(sequence)
            sequences.append(preds.flatten())
            speed_vals.append(speeds[frame_num-1])
        yield np.array(sequences), np.array(speed_vals)

In [63]:
ae_history = autoencoder.fit_generator(
    train_ae_gen, 
    steps_per_epoch=int(video_size*split/batch_size), 
    validation_data=val_ae_gen, 
    validation_steps=int(video_size*(1-split)/batch_size),
    epochs=epochs,
    verbose=True,
    callbacks=[ModelCheckpoint('./data/weights.hdf5',save_best_only=True)]
)

Epoch 1/10
1020/1020 [==============================] - 34s 33ms/step - loss: 4.3315 - val_loss: 0.6748
Epoch 2/10
1020/1020 [==============================] - 28s 28ms/step - loss: 0.3467 - val_loss: 0.2715
Epoch 3/10
1020/1020 [==============================] - 28s 28ms/step - loss: 0.2472 - val_loss: 0.2513
Epoch 4/10
1020/1020 [==============================] - 28s 28ms/step - loss: 0.2331 - val_loss: 0.2446
Epoch 5/10
1020/1020 [==============================] - 28s 28ms/step - loss: 0.2271 - val_loss: 0.2446
Epoch 6/10
1020/1020 [==============================] - 28s 28ms/step - loss: 0.2245 - val_loss: 0.2410
Epoch 7/10
1020/1020 [==============================] - 28s 28ms/step - loss: 0.2213 - val_loss: 0.2370
Epoch 8/10
1020/1020 [==============================] - 28s 28ms/step - loss: 0.2191 - val_loss: 0.2365
Epoch 9/10
1020/1020 [==============================] - 28s 28ms/step - loss: 0.2171 - val_loss: 0.2348
Epoch 10/10
1020/1020 [==============================] - 30s 29m

In [68]:
train_re_gen = concat_speed_generator(video[:int(video_size*split)], speeds[:int(video_size*split)], batch_size, reg_seq, encoder)
val_re_gen = concat_speed_generator(video[int(video_size*split):], speeds[int(video_size*split):], batch_size, reg_seq, encoder)
#print(train_re_gen.__next__())

In [69]:
reg_history = regression.fit_generator(
    train_re_gen, 
    steps_per_epoch=int(video_size*split/batch_size), 
    validation_data=val_re_gen, 
    validation_steps=int(video_size*(1-split)/batch_size),
    epochs=epochs,
    verbose=True,
    callbacks=[ModelCheckpoint('./data/reg_weights.hdf5',save_best_only=True)]
)

Epoch 1/10
1020/1020 [==============================] - 71s 69ms/step - loss: 47.1924 - val_loss: 26.0375
Epoch 2/10
1020/1020 [==============================] - 66s 65ms/step - loss: 23.0454 - val_loss: 21.7362
Epoch 3/10
1020/1020 [==============================] - 65s 64ms/step - loss: 20.8579 - val_loss: 22.8073
Epoch 4/10
1020/1020 [==============================] - 66s 64ms/step - loss: 19.7924 - val_loss: 22.5323
Epoch 5/10
1020/1020 [==============================] - 66s 65ms/step - loss: 19.0539 - val_loss: 21.6352
Epoch 6/10
1020/1020 [==============================] - 66s 65ms/step - loss: 18.4049 - val_loss: 21.8586
Epoch 7/10
1020/1020 [==============================] - 69s 67ms/step - loss: 18.0914 - val_loss: 21.0769
Epoch 8/10
1020/1020 [==============================] - 70s 68ms/step - loss: 18.0437 - val_loss: 22.7248
Epoch 9/10
1020/1020 [==============================] - 66s 64ms/step - loss: 17.6389 - val_loss: 21.5775
Epoch 10/10
1020/1020 [=======================

In [ ]:
# Plot the training loss against the validation loss
plt.plot(reg_history.history['loss'])
plt.plot(reg_history.history['val_loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Loss', 'Validation Loss'])
plt.savefig(fname='./data/lossplot')
plt.show()

plt.clf()

In [ ]:
def denseflow_as_image(x):
  x = x.astype(np.uint8)
  x = cv.cvtColor(x,cv.COLOR_HSV2BGR)
  return x

def show_grid_of_images(x):
  plt.figure(figsize=(50,5))
  for i in range(len(x)):
    plt.subplot(1,len(x),i+1)
    plt.imshow(denseflow_as_image(x[i]))
    plt.axis('off')

  plt.show()
  
#import imageio
'''
def display_gif(sequence):
  sequence = map(denseflow_as_image,sequence)
  imageio.mimsave('test.gif', sequence)

  from IPython.display import Image
  with open('test.gif','rb') as file:
      display(Image(file.read()))'''

#### MAKE THIS PRINT OUT THE EXPECTED IMG FOR NEXT FRAME
ims, _ = next(train_ae_gen)
preds = autoencoder.predict(ims)
show_grid_of_images(ims[1]*255)
show_grid_of_images([preds[1]*255])